In [1]:


import os, shutil, glob
os.environ["CUDA_VISIBLE_DEVICES"] = "1" # if we have only one GPU, otherwise "0, 1, etc"
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4038589676390835748
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 11154255222417196555
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13893090639895476959
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10943175066
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9850534990050127602
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:08:00.0, compute capability: 6.1"
]


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd
from PIL import Image
import random
from scipy import interp
from itertools import cycle
from sklearn.metrics import roc_curve, auc, roc_auc_score

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# TensorFlow and Keras
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.regularizers import l2
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import utils 

# import keras
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import metrics
from tensorflow.keras import optimizers 

print(tf.__version__)

#scratch-shared/maris/tests/labels.csv
#scratch-shared/maris/tests/MiCoPh_Image_0_255
#scratch-shared/maris/tests/MiSyHo_Image_0_255

2.1.0


In [3]:
basedir1 = input("Enter first directory name of images: ") # e.g.  /home/jovyan/scratch-shared/maris/tests/MiCoPh_Image_0_255

Enter first directory name of images:  /home/jovyan/scratch-shared/maris/tests/MiCoPh_Image_0_255


In [4]:
print(len(os.listdir(basedir1)))

12301


In [5]:
basedir2 = input("Enter second directory name of images: ") # e.g. /home/jovyan/scratch-shared/maris/tests/MiSyHo_Image_0_255

Enter second directory name of images:  /home/jovyan/scratch-shared/maris/tests/MiSyHo_Image_0_255


In [6]:
print(len(os.listdir(basedir2)))

12301


In [7]:
df=pd.read_csv('/home/jovyan/scratch-shared/maris/tests/labels.csv', sep=';')
df.columns = ['nr','plate','well',
                     'site','type','compoundname','target','concentration']
df.fillna("NotKnown", inplace = True)

df["filename"] = df["nr"].astype(str) + ".png"

df

,nr,plate,well,site,type,compoundname,target,concentration,filename
0,0,P009063,B02,1,compound,CBK013405,EGFR,10,0.png
1,1,P009063,B02,2,compound,CBK013405,EGFR,10,1.png
2,2,P009063,B02,3,compound,CBK013405,EGFR,10,2.png
3,3,P009063,B02,4,compound,CBK013405,EGFR,10,3.png
4,4,P009063,B02,5,compound,CBK013405,EGFR,10,4.png
...,...,...,...,...,...,...,...,...,...
12295,12295,P009083,G11,5,compound,CBK293913C,Rho,4,12295.png
12296,12296,P009083,G11,6,compound,CBK293913C,Rho,4,12296.png
12297,12297,P009083,G11,7,compound,CBK293913C,Rho,4,12297.png
12298,12298,P009083,G11,8,compound,CBK293913C,Rho,4,12298.png


In [8]:
activities = sorted(list(set(df['target']))) 
for a in activities:
    df[a] = [1 if a == x else 0 for x in df['target']]
print(activities)

['ALK', 'AMPK', 'ATM', 'ATM/ATR', 'Akt', 'Akt,CDK,PKA', 'Akt,MEK', 'Akt,PDK-1', 'Akt,S6kinase', 'AuroraKinase', 'AuroraKinase,FGFR,Bcr-Abl,c-RET,Src', 'AuroraKinase,VEGrFR', 'BTK', 'Bcr-Abl', 'Bcr-Abl,JAK,AuroraKinase', 'Bcr-Abl,VEGrFR,FGFR,PDGFR,Flt', 'CDK', 'CDK,AuroraKinase', 'CHK', 'Chk', 'DNA-PK,PI3K', 'DNA-PK,PI3K,mTOR', 'DNA-PK,mTOR,PDGF', 'DUB,Bcr-Abl', 'EGFR', 'EGFR,Flt,VEGrFR,HER2', 'EGFR,HER2', 'EGFR,mTOR', 'ERK', 'FAK', 'FGFR', 'FGFR,VEGrFR', 'FLT3', 'Flt', 'Flt,AuroraKinase,VEGrFR', 'Flt,Bcr-Abl,AuroraKinase', 'Flt,FGFR,PDGFR,VEGrFR,c-Kit', 'Flt,JAK,c-RET', 'FreeBase', 'GSK-3', 'HDAC,EGFR,HER2', 'HDAC,PI3K', 'HER2', 'HER2,EGFR', 'IA/ASB/IKK', 'IGF-1,ALK', 'IGF-1R', 'IKK', 'I¦ÊB/IKK', 'JAK', 'JAK,EGFR', 'JNK', 'LRRK2', 'MEK', 'NotKnown', 'Others', 'PAK', 'PDGFR', 'PDGFR,VEGrFR', 'PDGFR,c-Kit,Bcr-Abl', 'PDK-1', 'PDK-1,IKK', 'PI3K', 'PI3K,DNA-PK', 'PI3K,PKC,Src,Sirtuin', 'PI3K,VEGrFR', 'PI3K,mTOR', 'PI3K/Akt/mTOR', 'PKC', 'PLK', 'Pim', 'ProductCategoriesPI3K/Akt/mTORPI3KPI3K'

In [9]:
nrclasses = len(activities)
nrclasses

114

In [10]:
df = df[df.NotKnown != 1]
df = df.drop(columns='NotKnown')
activities.remove('NotKnown')
df

,nr,plate,well,site,type,compoundname,target,concentration,filename,ALK,...,"c-Met,ALK","c-Met,FGFR,PDGFR","c-Met,VEGrFR","c-Met,VEGrFR,Tie-2","c-Met,c-Kit,PDGFR,Flt,c-RET",c-Myc,mTOR,"mTOR,PI3K","mTOR,PI3K,Akt",p38MAPK
0,0,P009063,B02,1,compound,CBK013405,EGFR,10,0.png,0,...,0,0,0,0,0,0,0,0,0,0
1,1,P009063,B02,2,compound,CBK013405,EGFR,10,1.png,0,...,0,0,0,0,0,0,0,0,0,0
2,2,P009063,B02,3,compound,CBK013405,EGFR,10,2.png,0,...,0,0,0,0,0,0,0,0,0,0
3,3,P009063,B02,4,compound,CBK013405,EGFR,10,3.png,0,...,0,0,0,0,0,0,0,0,0,0
4,4,P009063,B02,5,compound,CBK013405,EGFR,10,4.png,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12295,12295,P009083,G11,5,compound,CBK293913C,Rho,4,12295.png,0,...,0,0,0,0,0,0,0,0,0,0
12296,12296,P009083,G11,6,compound,CBK293913C,Rho,4,12296.png,0,...,0,0,0,0,0,0,0,0,0,0
12297,12297,P009083,G11,7,compound,CBK293913C,Rho,4,12297.png,0,...,0,0,0,0,0,0,0,0,0,0
12298,12298,P009083,G11,8,compound,CBK293913C,Rho,4,12298.png,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
nrclasses = len(activities)
nrclasses

113

In [12]:
df['plate_well'] = df['plate'] + df['well']
welllist = sorted(list(set(df['plate_well'])))
print(len(welllist))
print(welllist[0:5])

def partitions (list_in, n):
    random.seed(1)
    random.shuffle(list_in)
    return [list_in[i::n] for i in range(n)]

chunks = partitions(welllist, 5) #split the dataset in 5 folds excluding 20% of images (a better way to validate would be excluding 20% of wells)
#print (chunks[0])
len(chunks)
df1=df[~df['plate_well'].isin(chunks[0])]
df2=df[df['plate_well'].isin(chunks[0])]
print(len(set(df1['nr'])))
print(len(df2))

1128
['P009063B02', 'P009063B04', 'P009063B05', 'P009063B06', 'P009063B07']
8838
2178


In [24]:
df1

,index,nr,plate,well,site,type,compoundname,target,concentration,filename,...,"c-Met,FGFR,PDGFR","c-Met,VEGrFR","c-Met,VEGrFR,Tie-2","c-Met,c-Kit,PDGFR,Flt,c-RET",c-Myc,mTOR,"mTOR,PI3K","mTOR,PI3K,Akt",p38MAPK,plate_well
0,0,0,P009063,B02,1,compound,CBK013405,EGFR,10,0.png,...,0,0,0,0,0,0,0,0,0,P009063B02
1,1,1,P009063,B02,2,compound,CBK013405,EGFR,10,1.png,...,0,0,0,0,0,0,0,0,0,P009063B02
2,2,2,P009063,B02,3,compound,CBK013405,EGFR,10,2.png,...,0,0,0,0,0,0,0,0,0,P009063B02
3,3,3,P009063,B02,4,compound,CBK013405,EGFR,10,3.png,...,0,0,0,0,0,0,0,0,0,P009063B02
4,4,4,P009063,B02,5,compound,CBK013405,EGFR,10,4.png,...,0,0,0,0,0,0,0,0,0,P009063B02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8833,12294,12294,P009083,G11,4,compound,CBK293913C,Rho,4,12294.png,...,0,0,0,0,0,0,0,0,0,P009083G11
8834,12295,12295,P009083,G11,5,compound,CBK293913C,Rho,4,12295.png,...,0,0,0,0,0,0,0,0,0,P009083G11
8835,12296,12296,P009083,G11,6,compound,CBK293913C,Rho,4,12296.png,...,0,0,0,0,0,0,0,0,0,P009083G11
8836,12297,12297,P009083,G11,7,compound,CBK293913C,Rho,4,12297.png,...,0,0,0,0,0,0,0,0,0,P009083G11


In [25]:
df2

,index,nr,plate,well,site,type,compoundname,target,concentration,filename,...,"c-Met,FGFR,PDGFR","c-Met,VEGrFR","c-Met,VEGrFR,Tie-2","c-Met,c-Kit,PDGFR,Flt,c-RET",c-Myc,mTOR,"mTOR,PI3K","mTOR,PI3K,Akt",p38MAPK,plate_well
0,108,108,P009063,C04,1,compound,CBK040892,EGFR,4,108.png,...,0,0,0,0,0,0,0,0,0,P009063C04
1,109,109,P009063,C04,2,compound,CBK040892,EGFR,4,109.png,...,0,0,0,0,0,0,0,0,0,P009063C04
2,110,110,P009063,C04,3,compound,CBK040892,EGFR,4,110.png,...,0,0,0,0,0,0,0,0,0,P009063C04
3,111,111,P009063,C04,4,compound,CBK040892,EGFR,4,111.png,...,0,0,0,0,0,0,0,0,0,P009063C04
4,112,112,P009063,C04,5,compound,CBK040892,EGFR,4,112.png,...,0,0,0,0,0,0,0,0,0,P009063C04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2173,12267,12267,P009083,G08,4,compound,CBK293913C,Rho,10,12267.png,...,0,0,0,0,0,0,0,0,0,P009083G08
2174,12268,12268,P009083,G08,5,compound,CBK293913C,Rho,10,12268.png,...,0,0,0,0,0,0,0,0,0,P009083G08
2175,12269,12269,P009083,G08,6,compound,CBK293913C,Rho,10,12269.png,...,0,0,0,0,0,0,0,0,0,P009083G08
2176,12270,12270,P009083,G08,7,compound,CBK293913C,Rho,10,12270.png,...,0,0,0,0,0,0,0,0,0,P009083G08


In [13]:
df1=df1.sort_values(by="plate_well") 
df1=df1.reset_index()

df2=df2.sort_values(by="plate_well")
df2=df2.reset_index()
df2.head()

,index,nr,plate,well,site,type,compoundname,target,concentration,filename,...,"c-Met,FGFR,PDGFR","c-Met,VEGrFR","c-Met,VEGrFR,Tie-2","c-Met,c-Kit,PDGFR,Flt,c-RET",c-Myc,mTOR,"mTOR,PI3K","mTOR,PI3K,Akt",p38MAPK,plate_well
0,108,108,P009063,C04,1,compound,CBK040892,EGFR,4,108.png,...,0,0,0,0,0,0,0,0,0,P009063C04
1,109,109,P009063,C04,2,compound,CBK040892,EGFR,4,109.png,...,0,0,0,0,0,0,0,0,0,P009063C04
2,110,110,P009063,C04,3,compound,CBK040892,EGFR,4,110.png,...,0,0,0,0,0,0,0,0,0,P009063C04
3,111,111,P009063,C04,4,compound,CBK040892,EGFR,4,111.png,...,0,0,0,0,0,0,0,0,0,P009063C04
4,112,112,P009063,C04,5,compound,CBK040892,EGFR,4,112.png,...,0,0,0,0,0,0,0,0,0,P009063C04


In [19]:
df1
#df1[activities].values

,index,nr,plate,well,site,type,compoundname,target,concentration,filename,...,"c-Met,FGFR,PDGFR","c-Met,VEGrFR","c-Met,VEGrFR,Tie-2","c-Met,c-Kit,PDGFR,Flt,c-RET",c-Myc,mTOR,"mTOR,PI3K","mTOR,PI3K,Akt",p38MAPK,plate_well
0,0,0,P009063,B02,1,compound,CBK013405,EGFR,10,0.png,...,0,0,0,0,0,0,0,0,0,P009063B02
1,1,1,P009063,B02,2,compound,CBK013405,EGFR,10,1.png,...,0,0,0,0,0,0,0,0,0,P009063B02
2,2,2,P009063,B02,3,compound,CBK013405,EGFR,10,2.png,...,0,0,0,0,0,0,0,0,0,P009063B02
3,3,3,P009063,B02,4,compound,CBK013405,EGFR,10,3.png,...,0,0,0,0,0,0,0,0,0,P009063B02
4,4,4,P009063,B02,5,compound,CBK013405,EGFR,10,4.png,...,0,0,0,0,0,0,0,0,0,P009063B02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8833,12294,12294,P009083,G11,4,compound,CBK293913C,Rho,4,12294.png,...,0,0,0,0,0,0,0,0,0,P009083G11
8834,12295,12295,P009083,G11,5,compound,CBK293913C,Rho,4,12295.png,...,0,0,0,0,0,0,0,0,0,P009083G11
8835,12296,12296,P009083,G11,6,compound,CBK293913C,Rho,4,12296.png,...,0,0,0,0,0,0,0,0,0,P009083G11
8836,12297,12297,P009083,G11,7,compound,CBK293913C,Rho,4,12297.png,...,0,0,0,0,0,0,0,0,0,P009083G11


In [21]:
df2
#df2[activities].values

,index,nr,plate,well,site,type,compoundname,target,concentration,filename,...,"c-Met,FGFR,PDGFR","c-Met,VEGrFR","c-Met,VEGrFR,Tie-2","c-Met,c-Kit,PDGFR,Flt,c-RET",c-Myc,mTOR,"mTOR,PI3K","mTOR,PI3K,Akt",p38MAPK,plate_well
0,108,108,P009063,C04,1,compound,CBK040892,EGFR,4,108.png,...,0,0,0,0,0,0,0,0,0,P009063C04
1,109,109,P009063,C04,2,compound,CBK040892,EGFR,4,109.png,...,0,0,0,0,0,0,0,0,0,P009063C04
2,110,110,P009063,C04,3,compound,CBK040892,EGFR,4,110.png,...,0,0,0,0,0,0,0,0,0,P009063C04
3,111,111,P009063,C04,4,compound,CBK040892,EGFR,4,111.png,...,0,0,0,0,0,0,0,0,0,P009063C04
4,112,112,P009063,C04,5,compound,CBK040892,EGFR,4,112.png,...,0,0,0,0,0,0,0,0,0,P009063C04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2173,12267,12267,P009083,G08,4,compound,CBK293913C,Rho,10,12267.png,...,0,0,0,0,0,0,0,0,0,P009083G08
2174,12268,12268,P009083,G08,5,compound,CBK293913C,Rho,10,12268.png,...,0,0,0,0,0,0,0,0,0,P009083G08
2175,12269,12269,P009083,G08,6,compound,CBK293913C,Rho,10,12269.png,...,0,0,0,0,0,0,0,0,0,P009083G08
2176,12270,12270,P009083,G08,7,compound,CBK293913C,Rho,10,12270.png,...,0,0,0,0,0,0,0,0,0,P009083G08


In [16]:
def calculating_class_weights(y_true):
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight('balanced', [0.,1.], y_true[:, i])
    return weights

def get_weighted_loss(weights):
    def weighted_loss(y_true, y_pred):
        return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)
    return weighted_loss

def weighted_loss(y_true, y_pred):
    return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)

import tensorflow.keras.losses
keras.losses.weighted_loss = weighted_loss

class_weights = calculating_class_weights(df1[activities].values)
print (class_weights)

scaling = 0 # some number between 0 and 1; if 0, no scaling is performed
class_weights_scaled = class_weights**scaling
print (class_weights_scaled) 

ValueError: classes should have valid labels that are in y

In [26]:
datagen= ImageDataGenerator(rescale=(1./255), horizontal_flip=True, vertical_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255.)

classes = None
#target_size = 270, 270
target_size = 224, 224 #input size for MobileNet and MobileNetV2
x_col = 'filename'
y_col = activities

train_generator1=datagen.flow_from_dataframe(
dataframe=df1,
directory=basedir1,
x_col="filename",
has_ext=True,
y_col=y_col,
batch_size=32,
seed=42,
shuffle=True,
class_mode="other",
classes = classes,
interpolation = "lanczos",
target_size=target_size)

validation_generator1=datagen.flow_from_dataframe(
dataframe=df2,
directory=basedir1,
x_col="filename",
has_ext=True,
y_col=y_col,
batch_size=1,
seed=42,
shuffle=False,
class_mode="other",
classes=classes,
interpolation = "lanczos",
target_size=target_size)

test_generator1=datagen.flow_from_dataframe(
dataframe=df2,
directory=basedir1,
x_col="filename",
has_ext=True,
y_col=y_col,
batch_size=1,
seed=42,
shuffle=False,
class_mode="other",
interpolation = "lanczos",
target_size=target_size)

train_generator2=datagen.flow_from_dataframe(
dataframe=df1,
directory=basedir2,
x_col="filename",
has_ext=True,
y_col=y_col,
batch_size=32,
seed=42,
shuffle=True,
class_mode="other",
classes=classes,
interpolation = "lanczos",
target_size=target_size)

validation_generator2=datagen.flow_from_dataframe(
dataframe=df2,
directory=basedir2,
x_col="filename",
has_ext=True,
y_col=y_col,
batch_size=1,
seed=42,
shuffle=False,
class_mode="other",
classes=classes,
interpolation = "lanczos",
target_size=target_size)

test_generator2=datagen.flow_from_dataframe(
dataframe=df2,
directory=basedir2,
x_col="filename",
has_ext=True,
y_col=y_col,
batch_size=1,
seed=42,
shuffle=False,
class_mode="other",
interpolation = "lanczos",
target_size=target_size)

Found 8838 validated image filenames.
Found 2178 validated image filenames.
Found 2178 validated image filenames.
Found 8838 validated image filenames.
Found 2178 validated image filenames.
Found 2178 validated image filenames.


In [27]:
def doubleGenerator(generator1,generator2): 
    while True:
        for (x1,y1),(x2,y2) in zip(generator1,generator2):
            yield (np.concatenate((x1,x2[:,:,:,1:]), axis=3),y1)

In [29]:
def doubleGenerator(generator1,generator2): #for 2 networks with 3 channels each
    while True:
        for (x1,y1),(x2,y2) in zip(generator1,generator2):
            yield ([x1,x2],y1)

In [32]:
pretrained_model1 = tf.keras.applications.ResNet50(input_shape=(224,224,3),
                                            include_top=False,
                                            weights='imagenet')

In [33]:
pretrained_model1.trainable = False
#pretrained_model1.summary()

In [34]:
pretrained_model2 = tf.keras.applications.ResNet50(input_shape=(224,224,3),
                                            include_top=False,
                                            weights='imagenet')

In [35]:
pretrained_model1.trainable = False
#pretrained_model1.summary()

In [37]:
last_output1 = pretrained_model1.output
last_output2 = pretrained_model2.output
print ('Pre-trained network 1: last layer output shape:', last_output1.shape)
print ('Pre-trained network 2: last layer output shape:', last_output2.shape)

Pre-trained network 1: last layer output shape: (None, 7, 7, 2048)
Pre-trained network 2: last layer output shape: (None, 7, 7, 2048)


In [38]:
x1 = layers.GlobalAveragePooling2D(name='avg_pool1')(last_output1)
x2 = layers.GlobalAveragePooling2D(name='avg_pool2')(last_output2)

In [39]:
#Concatenate dense layers of the two networks
xmerged = layers.concatenate(axis=-1, inputs=[x1, x2])
# Add a dropout rate of 0.5
xmerged = layers.Dropout(0.5)(xmerged)
x = layers.Dense(2048, activation='relu')(xmerged) # is it needed?

# Create output layer with n_classes nodes and sigmoid activation
output = layers.Dense(nrclasses, activation='sigmoid')(x) 

In [40]:
output

<tf.Tensor 'dense_1/Identity:0' shape=(None, 113) dtype=float32>

In [43]:
print ('Pre-trained network 1: input:', pretrained_model1.input)
print ('Pre-trained network 2: input:', pretrained_model2.input)

Pre-trained network 1: input: Tensor("input_2:0", shape=(None, 224, 224, 3), dtype=float32)
Pre-trained network 2: input: Tensor("input_3:0", shape=(None, 224, 224, 3), dtype=float32)


In [42]:
model = models.Model(inputs=[pretrained_model1.input, pretrained_model2.input], outputs=[output])

try:
    model = multi_gpu_model(model, gpus=2)
    print ('Using 2 GPUs')
except:
    print ("No more than 1 GPU available")
    
model.compile(loss=keras.losses.categorical_crossentropy,  optimizer=keras.optimizers.Adam(lr=0.001), metrics=['acc'])

ValueError: The name "conv1_pad" is used 2 times in the model. All layer names should be unique.